In [1]:
#import os, request, threading, urllib.request, urllib.error, urllib.parse, time
import os
import requests
import threading
import urllib.request, urllib.error, urllib.parse
import time

#mendifinisikan url yang merupakan tempat gambar yang akan di download
url = "https://apod.nasa.gov/apod/image/1901/LOmbradellaTerraFinazzi.jpg"


def buildRange(value, numsplits):
    lst = []
    for i in range(numsplits):
        if i == 0:
            lst.append('%s-%s' % (i, int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
        else:
            lst.append('%s-%s' % (int(round(1 + i * value/(numsplits*1.0),0)), int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
    return lst

class SplitBufferThreads(threading.Thread):
    """ Splits the buffer to ny number of threads
        thereby, concurrently downloading through
        ny number of threads.
    """
    #untuk mendapatkan konten dan ukuran konten
    def __init__(self, url, byteRange):
        super(SplitBufferThreads, self).__init__()
        self.__url = url
        self.__byteRange = byteRange
        self.req = None

    #untuk menghubungkan ke URL
    def run(self):
        self.req = urllib.request.Request(self.__url,  headers={'Range': 'bytes=%s' % self.__byteRange})

    #untuk mendapatkan data konten
    def getFileData(self):
        return urllib.request.urlopen(self.req).read()


def main(url=None, splitBy=3):
    #waktu download
    start_time = time.time()
    #jika url tidak terdefinisi/tidak sesuai
    if not url:
        print("Please Enter some url to begin download.")
        return

    fileName = url.split('/')[-1]
    #mendefinisikan ukuran file yang di download
    sizeInBytes = requests.head(url, headers={'Accept-Encoding': 'identity'}).headers.get('content-length', None)
    print("%s bytes to download." % sizeInBytes)
    #jika ukuran file tidak sesuai, maka muncul size cannot be determined
    if not sizeInBytes:
        print("Size cannot be determined.")
        return

    dataLst = []
    for idx in range(splitBy):
        byteRange = buildRange(int(sizeInBytes), splitBy)[idx]
        bufTh = SplitBufferThreads(url, byteRange)
        bufTh.start()
        bufTh.join()
        dataLst.append(bufTh.getFileData())

    content = b''.join(dataLst)

    #untuk menyimpan file ke komputer kita masing-masing
    if dataLst:
        if os.path.exists(fileName):
            os.remove(fileName)
        print("--- %s seconds ---" % str(time.time() - start_time))
        with open(fileName, 'wb') as fh:
            fh.write(content)
        print("Finished Writing file %s" % fileName)

#jalankan menu utama
if __name__ == '__main__':
    main(url)

3670260 bytes to download.
--- 157.30938076972961 seconds ---
Finished Writing file LOmbradellaTerraFinazzi.jpg
